# Hello !
## I'm here to help you to analyse data from Eden

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import glob
mpl.rc_file_defaults()

In [ ]:
# Folder in which you saved your files (by default, they are in build)
folder = "../build/"

In [ ]:
# Display list of available files
imsort = sorted(glob.glob(folder+'output*'),
                key=lambda f: int(''.join(filter(str.isdigit, f))))
imsort = [sort[9:] for sort in imsort]
imsort

In [ ]:
files = [
    'outputpor_M_10.out',
    'outputpor_M_15.out',
    'outputpor_M_25.out',
    'outputpor_M_50.out',
    'outputpor_M_75.out',
    'outputpor_M_100.out',
    'outputpor_M_150.out',
    'outputpor_M_200.out',
    'outputpor_M_250.out',
    'outputpor_M_300.out'
]

In [ ]:
# Create tuple of index and label
print("Reading data...")
try:
    # Check if files is a list
    if type(files) is not list:
        files = [files]

    # Add folder path
    files = [folder+file for file in files]

    # Create dataframe indices and labels
    with open(files[0], 'r') as f:
        ligne = f.readline().split()
        indices = [line.split("_")[0] for line in ligne]
        label = {key: r"$"+line.replace("_", " ")
                 .replace(" ", "$ $")+"$" for (key, line) in zip(indices, ligne)}

    # Create a list of dataframes
    data = [pd.read_csv(file, delim_whitespace=True, names=indices, skiprows=1)
            for file in files]
    print("Ok\n")
    print("List of available columns:")
    print(data[0].columns.tolist())
except:
    print("Problem with files")

# minmax function for auto-adjust and logscale axes
def minmax(data, columns, roundpower):
    def floor_power_of_10(n):
        return np.sign(n)*10**np.floor(np.math.log(np.abs(n), 10)) if n != 0 else 0

    def ceil_power_of_10(n):
        return np.sign(n)*10**np.ceil(np.math.log(np.abs(n), 10)) if n != 0 else 0

    minimum = min([datai.loc[:, columns].min() for datai in data])
    maximum = max([datai.loc[:, columns].max() for datai in data])

    if roundpower:
        minimum, maximum = floor_power_of_10(
            minimum), ceil_power_of_10(maximum)
    return minimum, maximum

# Everything here should remain untouched

In [ ]:
# Modify this cell to plot what you want !

# Axis
# t, r, mass, filfac, size, St, cg, sigma, rhog,dustfrac, vrel, omegak, drdt, dmdt, drag_reg
x = "r"
y = "size"
z = "filfac"

# Z axis option
z_axe_on = True
round_power = True

# Auto-adjust min-max
aax = True
aay = True
aaz = True

# Adjust min-max manually
xmin, xmax = 1, 300
ymin, ymax = 1e-7, 1e5
zmin, zmax = 1e-5, 1

# Logscale
logx = True
logy = True
logz = True
loglinval = 1e-5  # Symlognorm linthresh & linscale value

# Fontsize
fsize = 14
pointsize = 0.1

# Legend box location
location = "lower left"

# cmap
cmap = 'rainbow'

# Figure options
figname = x+'-'+y+'-'+z if z_axe_on is True else x+'-'+y
figname = 'fig_'+figname+'_fragmod.png'

figsave = False

In [ ]:
# Check if axes are correct
if (x not in label or y not in label or (z_axe_on is True and z not in label)):
    print("Error, quantities not found, use")
else:
    print("x,y,z axes Ok")

In [ ]:
print("Creating figure")
fig, axes = plt.subplots(1, 1, sharex='col', sharey='row',
                         gridspec_kw={'hspace': 0, 'wspace': 0, 'left': 0.11,
                                      'right': 0.99, 'bottom': 0.1, 'top': .975},
                         figsize=(6.4, 4.8), dpi=300)

# Plot data depending on options
for datai in data:
    if z_axe_on:
        if aaz:
            minimum, maximum = minmax(data, z, round_power)
            if logz:
                line = axes.scatter(datai.loc[:, x], datai.loc[:, y], c=datai.loc[:, z],
                                    s=pointsize, norm=colors.SymLogNorm(linthresh=loglinval, linscale=loglinval,
                                                                        vmin=minimum, vmax=maximum, base=10), cmap=cmap)
            else:
                line = axes.scatter(datai.loc[:, x], datai.loc[:, y], c=datai.loc[:, z],
                                    s=pointsize, vmin=minimum, vmax=maximum, cmap=cmap)
        else:
            if logz:
                line = axes.scatter(datai.loc[:, x], datai.loc[:, y], c=datai.loc[:, z],
                                    s=pointsize, norm=colors.SymLogNorm(linthresh=loglinval, linscale=loglinval,
                                                                        vmin=zmin, vmax=zmax, base=10), cmap=cmap)
            else:
                line = axes.scatter(datai.loc[:, x], datai.loc[:, y], c=datai.loc[:, z],
                                    s=pointsize, vmin=zmin, vmax=zmax, cmap=cmap)
    else:
        line = axes.scatter(datai.loc[:, x], datai.loc[:, y], s=0.1)
    
    # Label for the different radii
    line.set_label(r"$R_0=$" + str(datai.r[0]) + " au")

# Colorbar
if z_axe_on:
    clb = fig.colorbar(line, pad=0.005)
    clb.set_label(label[z])

# Axes and labels
if logx:
    axes.set(xscale='log')
if logy:
    axes.set(yscale='log')
if not aax:
    axes.set(xlim=(xmin, xmax))
if not aay:
    axes.set(ylim=(ymin, ymax))

axes.set(xlabel=label[x])
axes.set(ylabel=label[y])
axes.legend(fontsize='small', loc=location)

# save figure
if figsave:
    plt.savefig(figname, bbox_inches='tight')
    print(figname+" saved")

print("Your figure ->")

plt.show()